In [357]:
#%pip install nltk
#%pip install pandas
#%pip install numpy
#%pip install scikit-learn
#%pip install spacy
#%pip install pymorphy2

In [358]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import spacy
import sklearn
import pymorphy2
from pymorphy2 import MorphAnalyzer
import string
import re

In [359]:
#define
nltk.download('stopwords')
nltk.download('punkt')
rus = "russian"
sw = stopwords.words(rus)
morph = MorphAnalyzer()
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
lancaster = LancasterStemmer()
snowball = SnowballStemmer(language=rus)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NitghtWay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [360]:
test_ds1_path = "C:\\course_work_git\\feature_engineere_NLP\\datasets\\test_ds1\\train.jsonl"

In [361]:
data = pd.read_json(test_ds1_path, lines=True)
data

,premise,label,hypothesis,verb,negation,genre,idx,no_negation
0,Сумма ущерба составила одну тысячу рублей. Уто...,entailment,Ранее местный житель совершал подобное правона...,судить,no_negation,kp,0,NaN
1,"Перебрасываясь словечками, они скользят глазам...",contradiction,Они что-то понимают,смотреть,no_negation,fiction,1,NaN
2,"— Разве что, — сказала она, — мы хотим где-ниб...",neutral,"Это “ Таганская ”, а не “ Тульская ”.",понять,no_negation,fiction,2,NaN
3,"Зима, наконец, показала свой характер.",contradiction,У зимы есть свой характер.,показать,no_negation,kp,3,NaN
4,ГуманностьБогаподобнымсценариемнепредполагаетс...,contradiction,Это должно быть просто.,сказать,no_negation,fiction,4,NaN
...,...,...,...,...,...,...,...,...
433,Тверские спортсмены показали отличный результа...,entailment,У тверских спортсменов пять новых медалей.,суметь,no_negation,kp,433,NaN
434,"Просто я не хочу с ним жить и, извини, больше ...",contradiction,Этого было вполне достаточно.,уверять,no_negation,fiction,434,NaN
435,"Второй аргумент: все читаем и видим, что надви...",neutral,Надвигается вторая волна кризиса.,видеть,negation,interfax,435,NaN
436,"— Да, сын. Здравствуй. — Ты можешь говорить, н...",neutral,Сын не занят,говорить,no_negation,fiction,436,NaN


In [362]:
type(data)

pandas.core.frame.DataFrame

In [363]:
data = data.drop(['label', 'hypothesis', 'verb','negation','genre','idx','no_negation'], axis=1)

In [364]:
data.head()

,premise
0,Сумма ущерба составила одну тысячу рублей. Уто...
1,"Перебрасываясь словечками, они скользят глазам..."
2,"— Разве что, — сказала она, — мы хотим где-ниб..."
3,"Зима, наконец, показала свой характер."
4,ГуманностьБогаподобнымсценариемнепредполагаетс...


In [365]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   premise  438 non-null    object
dtypes: object(1)
memory usage: 3.5+ KB


In [366]:
data.shape

(438, 1)

Вспомогательные методы

In [367]:
#напиши python функцию, которая принимает на вход массив и выводит его элементы в виде таблицы с шириной 5 элементов и выровняет их по ширине, высота таблицы будет зависеть от общего количества элементов в массиве
def print_table(array):
    table_width = 5
    for i in range(0, len(array), table_width):
        row = array[i:i + table_width]
        formatted_row = " ".join(["{:<10}".format(str(elem)) for elem in row])
        print(formatted_row)

In [368]:
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in 
    string.punctuation])

In [369]:
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text]) 
    # метод для удаления чисел

In [370]:
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text

In [371]:
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text) # 

Стемминг с использованием SnowBall

In [372]:
example_text = data['premise'][2]
print(type(example_text))
print(example_text)

<class 'str'>
— Разве что, — сказала она, — мы хотим где-нибудь выпить кофе. — Мне кажется, — сказал он, — что мы хотим. На следующей Он успел встать на эскалатор, когда понял, что это “Таганская”, а не “Тульская”.


In [373]:
def tokenize_snowball(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = remove_patterns(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                continue
            else:
                all_words.append(snowball.stem(w))
            
    return all_words

In [374]:
print_table(tokenize_snowball(example_text))

разв       сказа      хот        вып        коф       
мне        кажет      сказа      хот        на        
след       он         успел      встат      эскалатор 
поня       эт         таганск    тульск    


In [375]:
lancaster = LancasterStemmer()
def tokenize_lancaster(text):
    text = remove_notalpha(text)
    text = remove_numbers(text)
    text = remove_patterns(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                continue
            else:
                all_words.append(lancaster.stem(w))
    return all_words

In [376]:
print_table(tokenize_lancaster(example_text))

разве      сказала    хотим      выпить     кофе      
мне        кажется    сказал     хотим      на        
следующей  он         успел      встать     эскалатор 
понял      это        таганская  тульская  


Лемматизация с помощью PyMorphy2

In [377]:
def tokenize_morphy(text):
    text = remove_notalpha(text)
    text = remove_patterns(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language=rus)
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language=rus)
        for w in words:
            if w in sw:
                continue
            else:
                all_words.append(morph.normal_forms(w)[0])
    if len(all_words) > 2:
        #all_words = pd.Series(all_words)
        return all_words
    else:
        return None
    

In [378]:
print_table(tokenize_morphy(example_text))

разве      сказать    хотеть     выпить     кофе      
я          казаться   сказать    хотеть     на        
следующий  он         успеть     встать     эскалатор 
понять     это        таганский  тульский  


In [379]:
df_morphy = pd.DataFrame({'text':[tokenize_morphy(sample) for sample in data['premise']]})
df_snowball = pd.DataFrame({'text':[tokenize_snowball(sample) for sample in data['premise']]})
df_lancaster = pd.DataFrame({'text':[tokenize_lancaster(sample) for sample in data['premise']]})

In [380]:
df_morphy.head()

,text
0,"[сумма, ущерб, составить, один, тысяча, рубль,..."
1,"[перебрасываться, словечко, скользить, глаз, м..."
2,"[разве, сказать, хотеть, выпить, кофе, я, каза..."
3,"[зима, показать, свой, характер]"
4,[гуманностьбогаподобнымсценариемнепредполагать...


In [381]:
df_snowball.head()

,text
0,"[сумм, ущерб, состав, одн, тысяч, рубл, уточня..."
1,"[перебрасыв, словечк, скольз, глаз, мо, город,..."
2,"[разв, сказа, хот, вып, коф, мне, кажет, сказа..."
3,"[зим, показа, сво, характер]"
4,"[гуманностьбогаподобнымсценариемнепредполага, ..."


In [382]:
df_lancaster.head()

,text
0,"[сумма, ущерба, составила, одну, тысячу, рубле..."
1,"[перебрасываясь, словечками, скользят, глазами..."
2,"[разве, сказала, хотим, выпить, кофе, мне, каж..."
3,"[зима, показала, свой, характер]"
4,[гуманностьбогаподобнымсценариемнепредполагает...
